# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [224]:
import selenium

In [225]:
from selenium import webdriver

In [226]:
driver = webdriver.Chrome()

In [227]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages.

### How will you identify the checkbox to check it?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

- *TIP: I have a secret awesome way to do this, but you have to ask.*

In [228]:
check_box = driver.find_element_by_xpath('//*[@id="checkbox"]')
check_box.click()

In [229]:
enter_site = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter_site.click()

### How will you identify the button to select it, or the form to submit it?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

### Once you're on the next page, how will you click the "SEARCH LICENSE RECORDS" link or follow its URL?

In [230]:
search_license_records = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search_license_records.click()

### Now you're on the form page. How do you pick "Statewide" for the jurisdiction dropdown?

In [231]:
from selenium.webdriver.support.ui import Select

In [232]:
jurisdiction = driver.find_element_by_xpath('//*[@id="slcJurisdiction"]')
select = Select(jurisdiction)
select.select_by_visible_text('Statewide')

### How do you type "vap%" into the Trade Name field?

In [233]:
trade_name = driver.find_element_by_xpath('//*[@id="txtTradeName"]')
trade_name.click()

In [234]:
trade_name.send_keys("vap%")

### How do you click the submit button or submit the form?

In [235]:
submit = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit.click()

### How can you find and click the 'Next' button on the search results page?

In [236]:
# next = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
# next.click()

# Okay, let's scrape!

### Use Selenium to search for vape shops statewide, and then click through until it reaches the last page.

Don't scrape yet!

In [237]:
# count = 0

# while True:
#     try:
#         next = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
#         next.click()
#     except: 
#         break

### Use Selenium to scrape the first page of search results for vape shops statewide (well, we'll try).

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

I've included some BeautifulSoup code that might be helpful. If you use it, **ask me how I made it.** It's important.

In [238]:
import requests
from bs4 import BeautifulSoup
doc = BeautifulSoup(driver.page_source, 'html.parser')

In [239]:
# IF YOU ARE USING BEAUTIFULSOUP, HERE IS SOME SAMPLE CODE
# IF YOU USE THIS CODE, ASK ME HOW I MADE IT. IT'S IMPORTANT.
business_headers = doc.find_all('tr',class_='searchfieldtitle')
len(business_headers)

5

In [240]:
# You'll probably need to find specific tds inside
# of the rows instead of looking at the whole thing.

#while True:
 #   doc = BeautifulSoup(driver.page_source, 'html.parser')
  #  business_headers = doc.find_all('tr',class_='searchfieldtitle')
   # for header in business_headers:
    #    rows = header.find_next_siblings('tr')
     #   print("HEADER is", header.text.strip())
      #  print("ROW 0 IS", rows[0].text.strip())
       # print("ROW 1 IS", rows[1].text.strip())
       # print("ROW 2 IS", rows[2].text.strip())
       # print("ROW 3 IS", rows[3].text.strip())
        #print("----")
    #try:
     #   next = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
     #   next.click()
    #except: 
     #   break

### Save these into `vape-results.csv`

In [241]:
results = [] 
while True:
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    business_headers = doc.find_all('tr',class_='searchfieldtitle')
    for header in business_headers:
        current = {} 
        rows = header.find_next_siblings('tr')
        current['company'] = header.find_all('td')[1].text.strip()
        current['link'] = header.find_all('td')[2].find('a')
        current['name'] = rows[0].find_all('td')[1].text.strip()
        current['address'] = rows[1].find_all('td')[1].text.strip()
        current['license'] = rows[1].find_all('td')[2].text.strip()
        current['city'] = rows[2].find_all('td')[1].text.strip()
        current['county'] = rows[3].text.strip()
        results.append(current) 
    try:
        next_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
        next_button.click()
    except: 
        break    

In [242]:
print(results) 

[{'company': 'VAPE IT STORE II', 'link': <a href="pbLicenseDetail.jsp?owi=c9l4Hc%2FAuto%3D"><img alt="Click for Detail of VAPE IT STORE II" src="images/link_click-detail.gif"/></a>, 'name': 'AMIN NARGIS', 'address': '1015 S SALISBURY BLVD', 'license': 'License: 22173808', 'city': 'SALISBURY, MD 21801', 'county': 'Wicomico County'}, {'company': 'VAPE IT STORE I', 'link': <a href="pbLicenseDetail.jsp?owi=pe9i6XrCMFE%3D"><img alt="Click for Detail of VAPE IT STORE I" src="images/link_click-detail.gif"/></a>, 'name': 'AMIN NARGIS', 'address': '1724 N SALISBURY BLVD UNIT 2', 'license': 'License: 22173807', 'city': 'SALISBURY, MD 21801', 'county': 'Wicomico County'}, {'company': 'VAPEPAD THE', 'link': <a href="pbLicenseDetail.jsp?owi=lycWL7%2FzX%2F0%3D"><img alt="Click for Detail of VAPEPAD THE" src="images/link_click-detail.gif"/></a>, 'name': 'ANJ DISTRIBUTIONS LLC', 'address': '2299 JOHNS HOPKINS ROAD', 'license': 'License: 02104436', 'city': 'GAMBRILLS, MD 21054', 'county': 'Anne Arundel

In [243]:
import pandas as pd
df = pd.DataFrame(results)
df

,address,city,company,county,license,link,name
0,1015 S SALISBURY BLVD,"SALISBURY, MD 21801",VAPE IT STORE II,Wicomico County,License: 22173808,"<a href=""pbLicenseDetail.jsp?owi=c9l4Hc%2FAuto...",AMIN NARGIS
1,1724 N SALISBURY BLVD UNIT 2,"SALISBURY, MD 21801",VAPE IT STORE I,Wicomico County,License: 22173807,"<a href=""pbLicenseDetail.jsp?owi=pe9i6XrCMFE%3...",AMIN NARGIS
2,2299 JOHNS HOPKINS ROAD,"GAMBRILLS, MD 21054",VAPEPAD THE,Anne Arundel County,License: 02104436,"<a href=""pbLicenseDetail.jsp?owi=lycWL7%2FzX%2...",ANJ DISTRIBUTIONS LLC
3,110 S. PINEY RD,"CHESTER, MD 21619",VAPE FROG,Queen Anne's County,License: 17165957,"<a href=""pbLicenseDetail.jsp?owi=fDi%2F1AXSsHo...",COX TRADING COMPANY L L C
4,346 RITCHIE HIGHWAY,"SEVERNA PARK, MD 21146",VAPE FROG,Anne Arundel County,,None,COX TRADING LLC
5,185 MITCHELLS CHANCE RD,"EDGEWATER, MD 21037",VAPE LOFT (THE),Anne Arundel County,License: 02102408,"<a href=""pbLicenseDetail.jsp?owi=0O2QDqy%2BGjo...",DISBROW II EMERSON HARRINGTON
6,7104 MINSTREL UNIT #7,"COLUMBIA, MD 21045",VAPE N CIGAR,Howard County,License: 13141786,"<a href=""pbLicenseDetail.jsp?owi=K9U7pDIwbkU%3...",DISCOUNT TOBACCO ESSEX LLC
7,330 ONE FORTY VILLAGE ROAD,"WESTMINSTER, MD 21157",VAPE DOJO,Carroll County,License: 06126253,"<a href=""pbLicenseDetail.jsp?owi=G6rN8X7s%2BA4...",FAIRGROUND VILLAGE LLC
8,29890 THREE NOTCH ROAD,"CHARLOTTE HALL, MD 20622",VAPE HAVEN,St. Mary's County,,None,GRIMM JENNIFER
9,356 ROMANCOKE ROAD,"STEVENSVILLE, MD 21666",VAPE BIRD,Queen Anne's County,License: 17166688,"<a href=""pbLicenseDetail.jsp?owi=eEFXW3k6AEE%3...",HUTCH VAPES LLC


### Open `vape-results.csv` to make sure there aren't any extra weird columns

In [245]:
df.to_csv("../vape_results.csv", index=False)

## Use Selenium to scrape ALL pages of results, save the results into `vape-results-all.csv`.

In [ ]:
#this is happening above! 
